<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_01_4_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 1: Course Overview**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 1 Material

* Part 1.1: Course Overview [[Video]](https://www.youtube.com/watch?v=OVS-6s20Ms0) [[Notebook]](t81_559_class_01_1_overview.ipynb)
* Part 1.2: Generative AI Overview [[Video]](https://www.youtube.com/watch?v=ohmPaSsKhMs) [[Notebook]](t81_559_class_01_2_genai.ipynb)
* Part 1.3: Introduction to OpenAI [[Video]](https://www.youtube.com/watch?v=C2xyi2Cq-bU) [[Notebook]](t81_559_class_01_3_openai.ipynb)
* **Part 1.4: Introduction to LangChain** [[Video]](https://www.youtube.com/watch?v=qQI5AhaKxuI) [[Notebook]](t81_559_class_01_4_langchain.ipynb)
* Part 1.5: Prompt Engineering [[Video]](https://www.youtube.com/watch?v=_Uot1i5sIXo) [[Notebook]](t81_559_class_01_5_prompt_engineering.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab


# Part 1.4: Introduction to LangChain

One of the most intriguing and promising developments in the evolving landscape of language models and artificial intelligence is LangChain. This technology represents a significant leap forward in how we interact with and harness the capabilities of large language models (LLMs). As we delve into the intricacies of LangChain in this chapter, it's important to understand not just the technical underpinnings but also the user experience that makes it so revolutionary.

## LangChain Chat Conversation Format

To explore LangChain comprehensively, we will adopt a format that has become increasingly familiar and effective in LLMs: the chat conversation interface. This interactive style, reminiscent of how many of us communicate daily, offers a unique and accessible means to illustrate LangChain's capabilities, potential applications, and the nuances of its operation.

We begin by importing the components from the LangChain library to support a chat-style interface to OpenAI. We will use the ChatOpenAI interface for the OpenAI family of LLM models.

In [ ]:
# Conversation Style Inteface

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

The conversation format consists of arrays of chat entries of the following three types:

* **SystemMessage** - This class designates the system prompt that provides instructions to the AI on the nature of the conversation and hints and guidelines. Generally, there will be only one system message at the beginning of the array.
* **HumanMessage** - This class designates the chat messages from outside the LLM, typically the human user.
* **AIMessage** - This class designates the chat messages from the LLM as responses to the HumanMessage messages.

Here we see the chain to ask a simple question.

In [ ]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that concisely and accurately answers questions."
    ),
    HumanMessage(
        content="What is the capital of France?"
    ),
]

We now submit these messages and retrieve the output from the model. We will use gpt-4o-mini, which is good enough for this query. Further, we use a zero temperature; we are simply looking for a factual answer, and creativity is not a goal or concern.

In [ ]:
MODEL = 'gpt-5-mini'

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
  model=MODEL,
  temperature= 0.0,
  n= 1,
  max_tokens= 256)

print("Model response:")
output = llm.invoke(messages)
print(output.content)
print("-----------")
print(output.response_metadata)

Model response:
The capital of France is Paris.
-----------
{'token_usage': {'completion_tokens': 16, 'prompt_tokens': 31, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C5bCdujUreNPvy9AUeQY2CHrwrkfS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}


The model that LangChain returns to you returns additional metadata. This data shows the token usage, which might be useful for estimating the total cost expected from this query.

We can continue to grow this conversation if we wish. To do so, we added the model's response and another human question. Here, we will ask the model if it was sure about its last response.

In [ ]:
messages.append(output)
messages.append(HumanMessage(content="Are you sure, I think it was renamed for some reason?"))
for message in messages:
    print(f"{type(message).__name__} : {message.content}")

SystemMessage : You are a helpful assistant that concisely and accurately answers questions.
HumanMessage : What is the capital of France?
AIMessage : The capital of France is Paris.
HumanMessage : Are you sure, I think it was renamed for some reason?


We can submit the conversation array to the model and see its latest response.

In [ ]:
print("Model response:")
output = llm.invoke(messages)
print(output.content)

Model response:



## Asking a Single Question

If you wish to ask the model a single question, not as part of a conversation chain, you can pass a string to the model for a response.

In [ ]:
# complete

from langchain_openai import OpenAI, ChatOpenAI

MODEL = 'gpt-5-mini'

# Initialize the OpenAI LLM (Language Learning Model) with your API key
llm = ChatOpenAI(model=MODEL, temperature=0)

# Define the question
question = "What are the five largest cities in the USA by population?"

# Use Langchain to call the OpenAI API
# The method and parameters might differ based on the Langchain version
response = llm.invoke(question)

# Print the response
display(response.content)

'By the 2020 U.S. Census, the five largest U.S. cities by population were:\n1. New York, NY — 8,336,817  \n2. Los Angeles, CA — 3,898,747  \n3. Chicago, IL — 2,746,388  \n4. Houston, TX — 2,304,580  \n5. Phoenix, AZ — 1,608,139\n\nIf you want the most recent population estimates (e.g., 2022–2023 Census Bureau estimates), I can provide those instead.'

## Prompt Templates

LangChain allows you to create chains of operations typically performed as part of an LLM-enabled application. One of these operations is a prompt template, which allows you to insert text into a previously created prompt. In this example, we will create a prompt template that asks the model to create a random blog post title.

```
Return only the title of a blog post article title on the topic of {topic} in {language}
```

To accomplish this objective, we will use a **PromptTemplate** object.

In [ ]:
from langchain.prompts import PromptTemplate

topic = "pets for data scientists"
language = "english"

# Initialize the OpenAI LLM (Language Learning Model) with your API key
# Use higher temperature for greater creativity
llm = ChatOpenAI(model=MODEL, temperature=0.7)

# Define the prompt template
title_template = PromptTemplate(
    input_variables=['topic', 'language'],
    template='Return only the title of a blog post article title on the topic of {topic} in {language}'
)

# Use RunnableSequence for chaining
title_chain = title_template | llm #RunnableSequence(steps=[title_template, llm])

# Invoke the chain with inputs
response = title_chain.invoke({'topic': topic, 'language': language})
print(response.content)

How My Cat Helped Me Tune Hyperparameters: Pet-Powered Lessons for Data Scientists


## Create a Simple Sequential Chain

We will now use LangChain to tie multiple LLM calls into a longer chain using the **SimpleSequentialChain** class. We will use two smaller chains to create a title and body text for a blog post. We begin by defining the two prompts we will use to construct this blog post. Also, note that we request that the LLM utilize [markdown](https://en.wikipedia.org/wiki/Markdown) to generate the actual blog post.


In [ ]:
# Create the two prompt templates
title_template = PromptTemplate( input_variables = ['topic'], template = 'Give me a blog post title on {topic} in English' )
article_template = PromptTemplate( input_variables = ['title'], template = 'Write a blog post for {title}, format in markdown.' )

We will create the first chain to generate the random title. Here, we allow the user to specify the topic. We use a higher temperature to increase the creativity of the title. We also use a simpler model to minimize cost for the relatively simple task of title selection.

In [ ]:
MODEL = 'gpt-5-mini'

# Create a chain to generate a random
llm = ChatOpenAI(model=MODEL, temperature=0.7)
title_chain = title_template | llm

Next, we compose the actual blog post; we will use a lower temperature to decrease creativity and cause the LLM to stick to factual information and avoid hallucinations. We also use a more complex model to provide a better article.

In [ ]:
MODEL2 = 'gpt-5'

# Create the article chain
llm2 = ChatOpenAI(model=MODEL2, temperature=0.1)
article_chain = article_template | llm2

Now, we combine these two chains into one. The input to the first chain will be the selected topic. The first chain will then output the title to the second chain, which will, in turn, output the actual article.

In [ ]:
# Create a complete chain to create a new blog post
#complete_chain=SimpleSequentialChain(chains=[title_chain, article_chain], verbose=True)

complete_chain = title_chain | article_chain

We can now display the final article. In this case, we requested an article on "photography," and displayed the final article's markdown.

In [ ]:
from IPython.display import display_markdown

article = complete_chain.invoke('photography')

The actual display of the markdown is handled by this code:

In [ ]:
display_markdown(article.content, raw=True)

# Mastering Light: How to Transform Everyday Scenes into Stunning Photographs

Light is the raw material of photography. Learn to see it—and shape it—and even the most ordinary scene becomes compelling. This guide gives you practical ways to read light, adapt fast, and create images with depth, mood, and clarity.

## What Light Really Does
- Shapes: Side light reveals texture; backlight carves edges; top light flattens.
- Sets mood: Warm tones feel intimate; cool tones feel calm or stark.
- Guides attention: Bright areas pull the eye; darker zones recede.
- Adds depth: Contrast between light and shadow creates three-dimensionality.

## The Four Variables (and how to use them)
1. Direction
   - Front: Low contrast, safe but flat. Good for color accuracy.
   - Side: Adds texture and drama. Great for landscapes, food, portraits.
   - Back: Glow, rim light, silhouettes. Watch for lens flare—use a hood or hand.
   - Top/Bottom: Harsh and unflattering. Move your subject or yourself.

2. Quality (hard vs. soft)
   - Hard: Small, distant sources (noon sun, bare bulb). Crisp shadows, high drama.
   - Soft: Large, close sources (window, overcast, diffuser). Gentle transitions, flattering skin.

3. Intensity
   - Control with exposure, ND filters, distance from light, or by moving into shade.
   - Expose for the highlights; lift shadows later if needed.

4. Color (white balance)
   - Typical Kelvin cheats: Daylight 5200–5600K, Shade 6500–7500K, Tungsten 2700–3200K, Golden Hour 4000–4800K, Blue Hour 9000–12000K.
   - Mixed light? Pick the dominant source and correct the rest (gels or post).

## Read the Scene in 30 Seconds
- Where is the brightest area?
- Where are the hardest edges in the shadows?
- What direction is the light coming from relative to your subject’s face/form?
- Is the color warm/cool? Mixed?
- What background stays clean at that angle?

Make a choice: move yourself, move the subject, add/subtract light, or change the time of day.

## Make Common Light Work for You
- Harsh midday sun
  - Put the sun behind your subject for rim light; expose for the highlights.
  - Use open shade or create shade with your body/hat.
  - Add “negative fill” (black side of a reflector, dark jacket) to restore contrast.
  - Try black-and-white to lean into graphic shapes.

- Overcast days
  - Soft, forgiving. Create direction by turning your subject toward/away from the brightest patch of sky.
  - Add a white reflector to open shadows or a black card to add definition.
  - Watch backgrounds; soft light can look flat without layers.

- Window light
  - Place the window at 45° for classic modeling. Turn the subject’s nose slightly toward the light (short lighting for slimming).
  - Control spill with curtains or a flag. Bounce with a white card/foil-lined board.
  - Set WB to 5000–6000K or use Auto and fine-tune in post.

- Golden hour and blue hour
  - Golden hour: warm, directional. Shoot side/backlight for glow; try a sunstar at f/16.
  - Blue hour: cool, even. Balance ambient with practical lights; use a tripod and longer exposures.

- Mixed lighting (office, street, interiors)
  - Decide on your hero light. Gel your flash to match or move your subject to simplify.
  - Shoot RAW for maximum flexibility.

## Simple Tools That Change Everything
- Collapsible reflector (white/silver/black) for fill or negative fill.
- 5-in-1 diffuser to soften harsh sun.
- Lens hood or your hand to tame flare.
- Small flash or LED for a kiss of fill; bounce off walls/ceilings.
- Polarizer to cut glare and deepen skies (mind the angle to the sun).

DIY: Foam board for bounce, black T-shirt for negative fill, sheer curtain as diffuser.

## Exposure and Color You Can Trust
- Use your histogram/zebras; protect highlights.
- Aperture Priority with exposure compensation is fast in changing light.
- Manual when the light is stable or for consistent series/portraits.
- Bracket (±2 EV) for high-contrast scenes; blend if needed.
- Shoot RAW. It’s your insurance for tricky color and dynamic range.

## Composition Meets Light
- Build around contrast: bright-on-dark or dark-on-bright subject isolation.
- Angle for separation: a small step can place your subject against a brighter/darker plane.
- Use leading lines that are lit; a lit line carries the eye.
- Clean your background; light plus chaos still looks chaotic.

## Mini Field Recipes
- Noon portrait without gear
  - Put the sun behind the subject; expose for skin.
  - Find open shade or use a building edge to block top light.
  - Add negative fill on the shadow side (dark jacket close to cheek).

- Moody window portrait
  - Subject one step from window; turn slightly away for short light.
  - Flag the far side with a dark board for contrast.
  - Spot meter for the bright cheek; let the background fall to shadow.

- Travel street scene at blue hour
  - Expose for the sky; let city lights glow.
  - Add a person crossing the brightest pool of light.
  - White balance around 8000–10000K to retain deep blues.

- Backlit leaves/food/details
  - Place light behind and slightly above; shoot toward the light.
  - Shield the lens from direct rays to avoid flare haze unless you want the glow.

## A One-Week Practice Plan
- Day 1: Photograph the same object in front, side, and backlight.
- Day 2: Hard vs. soft—bare bulb vs. diffused window.
- Day 3: Overcast textures—brick, tree bark, clothing.
- Day 4: Golden hour portraits—rim and short lighting.
- Day 5: Blue hour city frames—balance ambient and highlights.
- Day 6: Mixed light interior—choose and match a hero light.
- Day 7: Edit day—compare how light changed the story.

## Post-Processing That Honors the Light
- Start with white balance and exposure. Keep highlights believable.
- Local dodging/burning to guide the eye; avoid global clarity that crushes midtones.
- Use color grading to reinforce mood: warm mids for coziness, cooler shadows for calm.
- Dehaze sparingly; it boosts contrast but can look unnatural in skin.

## Common Mistakes
- Chasing subjects instead of chasing light.
- Ignoring backgrounds that fight your light.
- Overfilling faces until they look flat.
- Letting mixed color casts go unaddressed.

## A Quick Pre-Shot Checklist
- Where’s the brightest spot and the darkest?
- What direction and quality is the light?
- How will I isolate the subject using light contrast?
- Is my background helping?
- Are my highlights safe?

---

## Alternative Title Styles
- Beginner-friendly: “See the Light: Simple Tricks to Make Any Photo Look Better”
- Gear-focused: “Light Control 101: Reflectors, Diffusers, and Small Flash for Big Results”
- Portrait angle: “Shape a Face with Light: Everyday Setups for Flattering Portraits”
- Travel angle: “Chasing Light on the Road: Turn Ordinary Places into Memorable Photos”
- Creative angle: “Draw with Shadows: Crafting Mood from Common Light”
- Mobile angle: “Mastering Light with Your Phone: No Extra Gear Needed”

If you want, I can turn this into a printable one-page field checklist or tailor a version for portraits, travel, or phone-only shooting.